In [1]:
import numpy as np

In [2]:
samples = np.array([
    [0.3, -1.2, 1.5],   # Series A
    [2.0, 0.1, -0.5]    # Series B
])  # Shape: (2, 3)

permutations = np.array([
    [1, 2, 0],  # Series A's permutation
    [1, 2, 0]   # Series B's permutation
])  # Same shape

In [6]:
n_rows, n_cols = permutations.shape

In [7]:
aux_row_idx = np.arange(n_rows)[:, None] * n_cols

In [8]:
aux_row_idx

array([[0],
       [3]])

In [9]:
aux_row_idx = np.repeat(aux_row_idx, repeats=n_cols, axis=1)

In [10]:
aux_row_idx

array([[0, 0, 0],
       [3, 3, 3]])

In [11]:
permutate_idxs = permutations.flatten() + aux_row_idx.flatten()

In [12]:
permutate_idxs

array([1, 2, 0, 4, 5, 3])

In [14]:
permutated_samples = samples.flatten()
permutated_samples

array([ 0.3, -1.2,  1.5,  2. ,  0.1, -0.5])

In [15]:
permutated_samples = permutated_samples[permutate_idxs]
permutated_samples

array([-1.2,  1.5,  0.3,  0.1, -0.5,  2. ])

In [16]:
permutated_samples = permutated_samples.reshape(n_rows, n_cols)
permutated_samples

array([[-1.2,  1.5,  0.3],
       [ 0.1, -0.5,  2. ]])

In [28]:
def _permutate_samples( samples, permutations):
        """Permutate Samples

        Applies efficient vectorized permutation on the samples.

        **Parameters**<br>
        `samples`: np.array [series,samples], independent base samples.<br>
        `permutations`: np.array [series,samples], permutation ranks with wich
                  which `samples` dependence will be restored see `_obtain_ranks`.<br>

        **Returns**<br>
        `permutated_samples`: np.array.<br>
        """
        # Generate auxiliary and flat permutation indexes
        n_rows, n_cols = permutations.shape
        aux_row_idx = np.arange(n_rows)[:, None] * n_cols
        aux_row_idx = np.repeat(aux_row_idx, repeats=n_cols, axis=1)
        permutate_idxs = permutations.flatten() + aux_row_idx.flatten()

        # Apply flat permutation indexes and recover original shape
        permutated_samples = samples.flatten()
        permutated_samples = permutated_samples[permutate_idxs]
        permutated_samples = permutated_samples.reshape(n_rows, n_cols)
        return permutated_samples

In [25]:
prediction_samples = np.array([
    [  # Series A
        [10, 20, 30],  # t=0
        [15, 25, 35]   # t=1
    ],
    [  # Series B
        [40, 50, 60],  # t=0
        [45, 55, 65]   # t=1
    ]
])  # Shape: (2 series, 2 horizon, 3 samples)

permutations = np.array([
    [2, 0, 1],  # For Series A
    [1, 2, 0]   # For Series B
])

In [26]:
_, n_horizon, _ = prediction_samples.shape

In [20]:
permutated_prediction_samples = prediction_samples.copy()

In [29]:
for t in range(n_horizon):
    permutated_prediction_samples[:, t, :] = _permutate_samples(
        prediction_samples[:, t, :], permutations
    )

In [30]:
prediction_samples[:, t, :]

array([[15, 25, 35],
       [45, 55, 65]])

In [31]:
permutated_prediction_samples

array([[[30., 10., 20.],
        [35., 15., 25.]],

       [[50., 60., 40.],
        [55., 65., 45.]]])

In [33]:
y_insample = np.array([
    [110, 112],  # Total
    [60,  65],   # A
    [50,  47],   # B
])

y_hat_insample = np.array([
    [108, 110],  # Total
    [58,  63],   # A
    [50,  47],   # B
])

y_hat = np.array([
    [109, 111],  # Total
    [59,  64],   # A
    [50,  47],   # B
])

sigmah = np.array([
    [2, 2],      # Total
    [1.5, 1.5],  # A
    [1.2, 1.2],  # B
])


In [34]:
# Compute residuals and rank permutations
residuals = y_insample - y_hat_insample
residuals = residuals[:, np.isnan(residuals).sum(axis=0) == 0]
residuals

array([[2, 2],
       [2, 2],
       [0, 0]])

In [35]:
num_samples = 5

In [36]:
# Sample h step-ahead base marginal distributions
if num_samples is None:
    num_samples = residuals.shape[1]

In [37]:
num_samples

5

In [38]:
# Expand residuals to match num_samples [(a,b),T] -> [(a,b),num_samples]
rng = np.random.default_rng(40)
if num_samples > residuals.shape[1]:
    residuals_idxs = rng.choice(residuals.shape[1], size=num_samples)
else:
    residuals_idxs = rng.choice(
        residuals.shape[1], size=num_samples, replace=False
    )

In [39]:
residuals_idxs

array([1, 1, 0, 1, 0])

In [40]:
residuals = residuals[:, residuals_idxs]
residuals

array([[2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [0, 0, 0, 0, 0]])

In [41]:
residuals

array([[2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [0, 0, 0, 0, 0]])

In [42]:
def _obtain_ranks(array):
    """Vector ranks

    Efficiently obtain vector ranks.
    Example `array=[4,2,7,1]` -> `ranks=[2, 1, 3, 0]`.

    **Parameters**<br>
    `array`: np.array, matrix with floats or integers on which the
            ranks will be computed on the second dimension.<br>

    **Returns**<br>
    `ranks`: np.array, matrix with ranks along the second dimension.<br>
    """
    temp = array.argsort(axis=1)
    ranks = np.empty_like(temp)
    a_range = np.arange(temp.shape[1])
    for i_row in range(temp.shape[0]):
        ranks[i_row, temp[i_row, :]] = a_range
    return ranks

In [43]:
rank_permutations = _obtain_ranks(residuals)
rank_permutations

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [45]:
n_series, n_horizon = y_hat.shape

In [46]:
base_samples = np.array(
            [
                rng.normal(loc=m, scale=s, size=num_samples)
                for m, s in zip(y_hat.flatten(), sigmah.flatten())
            ]
        )

In [47]:
base_samples

array([[110.53731626, 106.92412907, 106.94792733, 108.21476148,
        111.77545789],
       [109.16509808, 109.60653796, 111.38661782, 111.28930431,
        111.77218752],
       [ 58.14411719,  57.59662455,  56.99672381,  59.49427972,
         58.68868815],
       [ 64.64015676,  64.06751449,  66.08571109,  64.87473998,
         64.24780695],
       [ 50.0528089 ,  49.38776753,  49.53520114,  49.61597819,
         51.82631025],
       [ 46.77298378,  46.28402848,  47.28980022,  47.86670571,
         45.64617933]])

In [49]:
base_samples = base_samples.reshape(n_series, n_horizon, num_samples)
base_samples

array([[[110.53731626, 106.92412907, 106.94792733, 108.21476148,
         111.77545789],
        [109.16509808, 109.60653796, 111.38661782, 111.28930431,
         111.77218752]],

       [[ 58.14411719,  57.59662455,  56.99672381,  59.49427972,
          58.68868815],
        [ 64.64015676,  64.06751449,  66.08571109,  64.87473998,
          64.24780695]],

       [[ 50.0528089 ,  49.38776753,  49.53520114,  49.61597819,
          51.82631025],
        [ 46.77298378,  46.28402848,  47.28980022,  47.86670571,
          45.64617933]]])

In [50]:
# Initialize PERMBU utility
rec_samples = base_samples.copy()

In [56]:
def _nonzero_indexes_by_row(M):
        return [np.nonzero(M[row, :])[0] for row in range(len(M))]

In [57]:
S = np.array([
    [1, 1, 0],  # Total = A + B
    [0, 1, 0],  # A
    [0, 0, 1],  # B
])

In [61]:
import numpy as np

def pad_to_max_length(arrays, pad_value=-1):
    max_len = max(len(a) for a in arrays)
    return np.vstack([
        np.pad(a, (0, max_len - len(a)), constant_values=pad_value)
        for a in arrays
    ])

#padded_hier_links = pad_to_max_length(_nonzero_indexes_by_row(S.T))

In [62]:
from sklearn.preprocessing import OneHotEncoder
# Initialize PERMBU utility
rec_samples = base_samples.copy()
try:
    encoder = OneHotEncoder(sparse_output=False, dtype=np.float64)
except TypeError:
    encoder = OneHotEncoder(sparse=False, dtype=np.float64)
#hier_links = np.vstack(_nonzero_indexes_by_row(S.T))
hier_links = pad_to_max_length(_nonzero_indexes_by_row(S.T))

In [66]:
hier_links

array([[ 0, -1],
       [ 0,  1],
       [ 2, -1]])

In [67]:
# BottomUp hierarchy traversing
hier_levels = hier_links.shape[1] - 1

In [69]:
for level_idx in reversed(range(hier_levels)):
    # Obtain aggregation matrix from parent/children links
    children_links = np.unique(hier_links[:, level_idx : level_idx + 2], axis=0)
    children_idxs = np.unique(children_links[:, 1])
    parent_idxs = np.unique(children_links[:, 0])
    Agg = encoder.fit_transform(children_links).T
    Agg = Agg[: len(parent_idxs), :]

In [72]:
hier_links[:, 0 : 0 + 2]

array([[ 0, -1],
       [ 0,  1],
       [ 2, -1]])

In [81]:
children_links = np.unique(hier_links[:, 0 : 0 + 2], axis=0)
children_links

array([[ 0, -1],
       [ 0,  1],
       [ 2, -1]])

In [82]:
children_idxs = np.unique(children_links[:, 1])
children_idxs

array([-1,  1])

In [83]:
parent_idxs = np.unique(children_links[:, 0])
parent_idxs

array([0, 2])

In [85]:
Agg = encoder.fit_transform(children_links) #.T
Agg

array([[1., 0., 1., 0.],
       [1., 0., 0., 1.],
       [0., 1., 1., 0.]])